In [10]:
# !pip install implicit

import numpy as np
import pandas as pd

from utils.data_transformation import get_train_csr_matrix, get_test_warm_data
from utils.evaluation import compute_metrics

from implicit.als import AlternatingLeastSquares

In [11]:
train_ratio, watched_pct_threshold = 0.8, -1
user_movies, users_rev_mapping, movies_rev_mapping = get_train_csr_matrix(train_ratio, watched_pct_threshold)

als_model = AlternatingLeastSquares(factors=128, regularization=0.05, iterations=50, random_state=42)
als_model.fit(user_movies)

  0%|          | 0/50 [00:00<?, ?it/s]

### Example of making recommendations by AlternatingLeastSquares

In [12]:
df_inter = pd.read_csv('data/processed/interactions.csv')
df_items = pd.read_csv('data/processed/items.csv')

train_size = int(train_ratio*len(df_inter))

users_mapping = {user_id: idx for idx, user_id in users_rev_mapping.items()}
movies_mapping = {movie_id: idx for idx, movie_id in movies_rev_mapping.items()}
id_df_idx_mapping = {id: idx for idx, id in df_items['item_id'].items()}

In [22]:
user_id = 148609
df_inter[df_inter['user_id'] == user_id].merge(df_items)

,user_id,item_id,last_watch_dt,watched_pct,content_type,title,genres,age_rating,keywords,views
0,148609,12395,2021-03-23,100.0,film,"Беги, мальчик, беги","боевики, драмы, биография, военные",16,"вторая мировая война, варшавское гетто, дети, ...",100
1,148609,13713,2021-04-01,0.0,film,Звёздные войны: Скрытая угроза,"боевики, фантастика, приключения",0,"пророчество, сенат, королева, надзиратель, гал...",444
2,148609,11885,2021-04-01,100.0,film,Хан Соло: Звёздные войны. Истории,"боевики, фантастика, приключения",12,"космический корабль, приквел, спин-офф, контра...",621
3,148609,13980,2021-04-11,100.0,film,Изгой-один: Звёздные войны. Истории.,"боевики, фантастика, приключения",16,"бунтарь, космический корабль, космическое сраж...",416


In [23]:
test_user_idx = users_mapping[user_id]
rec_movies_idx = als_model.recommend(test_user_idx, user_movies[test_user_idx], N=20)[0]

rec_movies_id = np.vectorize(movies_rev_mapping.get)(rec_movies_idx)
rec_movies_id = np.vectorize(id_df_idx_mapping.get)(rec_movies_id)
df_items.iloc[rec_movies_id].head().reset_index(drop=True)

,item_id,content_type,title,genres,age_rating,keywords,views
0,4702,film,Хищник,"боевики, фантастика, триллеры, приключения",16,"Центральная и Южная Америка, хищник, инопланет...",1164
1,8437,film,Капитан Марвел,"боевики, фантастика",16,"супергерой, основанный на комиксе, супергероин...",1228
2,13411,film,Вальгалла: Рагнарёк,"фэнтези, триллеры, приключения",12,"2019, дания, норвегия, швеция, исландия, вальг...",1111
3,12623,film,Пираты карибского моря: Проклятие чёрной жемчу...,"боевики, фэнтези, приключения",12,"экзотический остров, кузнец, торговая компания...",1288
4,12841,film,Стражи Галактики,"боевики, фантастика, приключения, комедии",12,"космический корабль, основанный на комиксе, ко...",1688


In [24]:
df_items.iloc[rec_movies_id]

,item_id,content_type,title,genres,age_rating,keywords,views
148,4702,film,Хищник,"боевики, фантастика, триллеры, приключения",16,"Центральная и Южная Америка, хищник, инопланет...",1164
143,8437,film,Капитан Марвел,"боевики, фантастика",16,"супергерой, основанный на комиксе, супергероин...",1228
167,13411,film,Вальгалла: Рагнарёк,"фэнтези, триллеры, приключения",12,"2019, дания, норвегия, швеция, исландия, вальг...",1111
135,12623,film,Пираты карибского моря: Проклятие чёрной жемчу...,"боевики, фэнтези, приключения",12,"экзотический остров, кузнец, торговая компания...",1288
93,12841,film,Стражи Галактики,"боевики, фантастика, приключения, комедии",12,"космический корабль, основанный на комиксе, ко...",1688
109,2237,film,Код 8,"боевики, фантастика",18,"торговля наркотиками, антиутопия, переделка, о...",1503
163,14264,film,Пираты Карибского Моря: Сундук Мертвеца,"боевики, фэнтези, приключения",12,"ведьма, гадалка, неволя, экзотический остров, ...",1121
149,5754,film,Доктор Стрэндж,"боевики, фантастика, фэнтези, приключения",16,"магия, время, обучение, супергерой, по мотивам...",1163
142,366,film,Послезавтра,"драмы, фантастика, триллеры, приключения",12,"спасение мира, библиотека, катаклизм, изменени...",1229
181,11348,film,Пираты Карибского моря: Мертвецы не рассказыва...,"боевики, фэнтези, приключения",16,"море, корабль, сиквел, артефакт, карта сокрови...",1033


### Metrics

In [16]:
test_data = get_test_warm_data(train_ratio, watch_threshold=1)

test_interactions = {}
for user_id, item_id in test_data[['user_id', 'item_id']].values:
    user_idx = users_mapping[user_id]
    if user_idx not in test_interactions:
        test_interactions[user_idx] = set()
    test_interactions[user_idx].add(movies_mapping[item_id])

users_idx = np.array(list(test_interactions.keys()))
recommendations = als_model.recommend(users_idx, user_movies[users_idx], N=20)[0]

precision, recall, map = compute_metrics(users_idx, test_interactions, recommendations)
precision, recall, map

(0.004749194281464787, 0.04535386163645981, 0.001087688424739371)